Лабораторна робота 4

1. Написати функцію пошуку випадкового простого числа з заданого інтервалу або
заданої довжини, використовуючи датчик випадкових чисел та тести перевірки на
простоту. В якості датчика випадкових чисел використовуйте вбудований генератор
псевдовипадкових чисел вашої мови програмування. В якості тесту перевірки на простоту
рекомендовано використовувати тест Міллера-Рабіна із попередніми пробними
діленнями. Тести необхідно реалізовувати власноруч, використання готових реалізацій
тестів не дозволяється.


In [62]:
import csv
import random
from math import gcd

def miller_rabin(p, k=5):
    if p <= 1:
        return False
    if p <= 3:
        return True
    if p % 2 == 0:
        return False

    s, d = 0, p - 1
    while d % 2 == 0:
        s += 1
        d //= 2

    for _ in range(k):
        x = random.randint(2, p - 2)

        if gcd(x, p) > 1:
            return False

        x_d = pow(x, d, p)
        if x_d == 1 or x_d == p - 1:
            continue

        is_composite = True
        for r in range(1, s):
            x_d = pow(x_d, 2, p)
            if x_d == p - 1:
                is_composite = False
                break
            if x_d == 1:
                return False

        if is_composite:
            return False

    return True

def random_prime_in_range(start, end):
    with open("numbers.csv", "a", newline="") as csvfile:
        while True:
            num = random.randint(start, end)
            if miller_rabin(num):
                return num
            else:
                    writer = csv.writer(csvfile)
                    writer.writerow([num])

def random_prime_with_bit_length(bits):
    start = 2**(bits - 1)
    end = 2**bits - 1
    return random_prime_in_range(start, end)

2. За допомогою цієї функції згенерувати дві пари простих чисел  p, q і p1, q1 довжини щонайменше 256 біт. При цьому пари чисел беруться так, щоб pq <= p1q1; p і q – прості числа для побудови ключів абонента А, p1 i q1 – абонента B.

In [60]:
def generate_prime_pairs(bits):
    while True:
        p = random_prime_with_bit_length(bits)
        q = random_prime_with_bit_length(bits)
        p1 = random_prime_with_bit_length(bits)
        q1 = random_prime_with_bit_length(bits)

        if p * q <= p1 * q1:
            return (p, q), (p1, q1)

(pre_p, pre_q), (pre_p1, pre_q1) = generate_prime_pairs(256)
print("Прості числа абонента A: p =", pre_p, ", q =", pre_q)
print("Прості числа абонента B: p1 =", pre_p1, ", q1 =", pre_q1)
print("Перевірка: pq =", pre_p * pre_q, "<= p1q1 =", pre_p1 * pre_q1)
print(pre_p * pre_q <= pre_p1 * pre_q1)

Прості числа абонента A: p = 87354766546536708125860530043437692278267889467788947559195344186410445075677 , q = 70969428545336947774909506929314876801436006294207101884960887041625768736037
Прості числа абонента B: p1 = 109844110988805784989510066311670915614827770681105434856028158772259005931803 , q1 = 61892729654457728640911458076385155627864595372413441094428276424293828469689
Перевірка: pq = 6199517862519027318773374730248631992208510972162208053511034285084524491506992403880606782403101235798360836003248829638941861024033533928858007102072049 <= p1q1 = 6798551865564405866302031189431336197941663666238815826799356373817831999870609882263345537738868882378356835100680207512408073103494561490962048986619267
True


3. Написати функцію генерації ключових пар для RSA. Після генерування функція повинна повертати та/або зберігати секретний ключ (d, p,q) та відкритий ключ  (n,e). За допомогою цієї функції побудувати схеми RSA для абонентів А і B – тобто, створити та зберегти для подальшого використання відкриті ключі (e,n) , (e1, n1) та секретні d і d1.

In [63]:
def mod_inverse(e, phi):
    old_r, r = e, phi
    old_s, s = 1, 0
    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
    if old_r != 1:
        raise ValueError("e не має оберненого за модулем phi")
    return old_s % phi

def generate_rsa_key_pair(p, q):
    n = p * q
    phi = (p - 1) * (q - 1)

    while True:
        e = random.randint(2, phi - 1)
        if gcd(e, phi) == 1:
            break

    d = mod_inverse(e, phi)
    return (e, n), (d, p, q)

public_key_A, private_key_A = generate_rsa_key_pair(pre_p, pre_q)
public_key_B, private_key_B = generate_rsa_key_pair(pre_p1, pre_q1)

print("Відкритий ключ абонента A (e, n):", public_key_A)
print("Секретний ключ абонента A (d, p, q):", private_key_A)
print()
print("Відкритий ключ абонента B (e1, n1):", public_key_B)
print("Секретний ключ абонента B (d1, p1, q1):", private_key_B)


Відкритий ключ абонента A (e, n): (2090717405462280846363333318198179891038308849994802498107708820113992554166815991248223180605239660836595859803647886917813644932511049381546178044082813, 6199517862519027318773374730248631992208510972162208053511034285084524491506992403880606782403101235798360836003248829638941861024033533928858007102072049)
Секретний ключ абонента A (d, p, q): (488911208777162550234952358952283809207194196489492087750160718660324427773825085113057532785115659963665570809352364039089252070941172088899893629721573, 87354766546536708125860530043437692278267889467788947559195344186410445075677, 70969428545336947774909506929314876801436006294207101884960887041625768736037)

Відкритий ключ абонента B (e1, n1): (5378017735986257894259697386674840738618879007551531753625999798464739705565118294898983107754484584287912401659757811261974254651471549589671867361653143, 679855186556440586630203118943133619794166366623881582679935637381783199987060988226334553773886888237835683

4. Написати програму шифрування, розшифрування і створення повідомлення з  цифровим підписом для абонентів А і B. Кожна з операцій (шифрування, розшифрування, створення цифрового підпису, перевірка цифрового підпису) повинна бути реалізована окремою процедурою, на вхід до якої повинні подаватись лише ті ключові дані, які  необхідні для її виконання.
За допомогою датчика випадкових чисел вибрати відкрите повідомлення M і
знайти криптограму для абонентів А и B, перевірити правильність розшифрування.  Скласти для А і B повідомлення з цифровим підписом і перевірити його.

In [64]:
def encrypt(message, public_key):
    return pow(message, public_key[0], public_key[1])

def decrypt(ciphertext, private_key):
    return pow(ciphertext, private_key[0], private_key[1] * private_key[2])

def sign(message, private_key):
    return pow(message, private_key[0], private_key[1] * private_key[2])

def verify(message, signature, public_key):
    return message == pow(signature, public_key[0], public_key[1])


5. За допомогою раніше написаних на попередніх етапах програм організувати роботу протоколу конфіденційного розсилання ключів з підтвердженням справжності по  відкритому каналу за допомогою алгоритму RSA. Протоколи роботи кожного учасника  (відправника та приймаючого) повинні бути реалізовані у вигляді окремих процедур, на  вхід до яких повинні подаватись лише ті ключові дані, які необхідні для виконання.
Перевірити роботу програм для випадково обраного ключа  0 < k < n.

In [65]:
def send_key(message, receiver_public_key, sender_private_key):
    signed_message = sign(message, sender_private_key)
    encrypted_signed_message = encrypt(signed_message, receiver_public_key)
    encrypted_message = encrypt(message, receiver_public_key)
    return encrypted_message, encrypted_signed_message

def receive_key(encrypted_message, encrypted_signed_message, receiver_private_key, sender_public_key):
    decrypted_message = decrypt(encrypted_message, receiver_private_key)
    signed_message = decrypt(encrypted_signed_message, receiver_private_key)
    if verify(decrypted_message, signed_message, sender_public_key):
        print("Перевірка підпису успішна")
        return decrypted_message
    else:
        raise ValueError("Невдала перевірка підпису")

k = random.randint(2, public_key_A[1] - 1)
print("Випадкове повідомлення k:", k)

encrypted_message_to_B, encrypted_signed_message_to_B  = send_key(k, public_key_B, private_key_A)
print("Зашифроване повідомлення:", encrypted_message_to_B)
print("Зашифроване підписане повідомлення:", encrypted_signed_message_to_B)

try:
    received_k = receive_key(encrypted_message_to_B, encrypted_signed_message_to_B, private_key_B, public_key_A)
    print("Отриманий ключ:", received_k)
except ValueError as e:
    print(e)

Випадкове повідомлення k: 4212129922692477871409065517171069466072456671266660037618142382787102334281078445984522432429084893263603942244071447884411535319882687025260369042191534
Зашифроване повідомлення: 850124687929945285026952277711881845013525663610290968609733682036741279941152758588070030467049900146237976866168108794108245425347883672677966955802585
Зашифроване підписане повідомлення: 472824198432174381351844382503329602334530639546915050605192621331795526308025365475759446891930802072807723309120491533900075357210830469730252111233477
Перевірка підпису успішна
Отриманий ключ: 4212129922692477871409065517171069466072456671266660037618142382787102334281078445984522432429084893263603942244071447884411535319882687025260369042191534
